**About** : This notebook is used to train models.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
device = torch.cuda.get_device_name(0)
print(device)

In [ ]:
import os
import sys
import glob
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *

pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 30)
pd.set_option('max_colwidth', 100)

In [ ]:
from training.main import k_fold

from util.logger import (
    prepare_log_folder,
    save_config,
    create_logger,
    init_neptune
)
from util.torch import init_distributed
from util.plots import *

from params import *
from data.dataset import *
from data.preparation import *
from data.transforms import get_transfos

from model_zoo.models import define_model
from training.losses import *

## Data
- Frame level label :
    - Ok for extravasion & bowel
    - Use seg for other classes

In [ ]:
df_extrav = prepare_extrav_data(DATA_PATH)

In [ ]:
df_patient, df_img = prepare_data(DATA_PATH)

In [ ]:
transforms = get_transfos(augment=False, resize=(384, 384), strength=5, crop=False)

dataset = AbdominalDataset(
    df_patient, #[df_patient["any_injury"] == 1].reset_index(drop=True).head(100),
    df_img,
    transforms=transforms,
    train=True,
    frames_chanel=0,
    n_frames=3,
    stride=1,
    use_soft_target=False,
    use_mask=False,
    use_crops=False,
)

In [ ]:
i = np.random.choice(len(dataset))
# i = 8784
img, y_img, y_patient = dataset[i]

if len(img.size()) == 4:
    plt.figure(figsize=(15, 5))
    for i, img_ in enumerate(img.numpy()):
        plt.subplot(1, img.size(0), i + 1)
        plt.imshow(img_.transpose(1, 2, 0))
        plt.axis(False)
    plt.show()
elif len(img.size()) == 3:
    plt.figure(figsize=(5, 5))
    plt.imshow(img.numpy().transpose(1, 2, 0))
    plt.axis(False)
    plt.show()

In [ ]:
# i = np.random.choice(len(dataset))
# img, y_img, y_patient = dataset[i]

# if len(img) == 4:
#     img, mask = img[1][None], img[-1]
#     plot_mask(img.numpy().transpose(1, 2, 0), mask.numpy())

In [ ]:
# for i in range(10):
# #     i = 0
#     img, y_img, y_patient = dataset[i]

#     plt.figure(figsize=(8, 8))
#     plt.imshow(img.numpy().transpose(1, 2, 0))
    
#     plt.title(f"#{i}\n{PATIENT_TARGETS} - {y_img}\n{IMG_TARGETS_EXTENDED} - {y_patient}")
#     plt.show()

# #     break

### Crop dataset

In [ ]:
transforms = get_transfos(augment=False, resize=(384, 384), strength=5, crop=False)

In [ ]:
dataset = AbdominalCropDataset(
    df_patient, #[df_patient["any_injury"] == 1].reset_index(drop=True).head(100),
    df_img,
    transforms=transforms,
    train=True,
    frames_chanel=1,
    n_frames=3,
    stride=3,
    use_mask=True
)

In [ ]:
img, y, _ = dataset[2]

if len(img.size()) == 4:
    plt.figure(figsize=(15, 5))
    for i, img_ in enumerate(img.numpy()):
        plt.subplot(1, img.size(0), i + 1)
        if img_.shape[0] == 3:
            plt.imshow(img_.transpose(1, 2, 0))
        else:
            plot_mask(img_[:3].transpose(1, 2, 0), img_[-1])
        plt.axis(False)
    plt.show()
else:
    plt.figure(figsize=(5, 5))
    plt.imshow(img.numpy().transpose(1, 2, 0))
    plt.axis(False)
    plt.show()

In [ ]:
img, y, _ = dataset[2]

if len(img.size()) == 4:
    plt.figure(figsize=(15, 5))
    for i, img_ in enumerate(img.numpy()):
        plt.subplot(1, img.size(0), i + 1)
        if img_.shape[0] == 3:
            plt.imshow(img_.transpose(1, 2, 0))
        else:
            plot_mask(img_[:3].transpose(1, 2, 0), img_[-1])
        plt.axis(False)
    plt.show()
else:
    plt.figure(figsize=(5, 5))
    plt.imshow(img.numpy().transpose(1, 2, 0))
    plt.axis(False)
    plt.show()

In [ ]:
# for i in tqdm(range(len(dataset))):
#     _ = dataset[i]

### Extrav Dataset

In [ ]:
transforms = get_transfos(augment=False, resize=(96, 96), strength=5, crop=False)

In [ ]:
dataset = AbdominalExtravDataset(
    df_extrav, #[df_patient["any_injury"] == 1].reset_index(drop=True).head(100),
    transforms=transforms,
    train=False,
    frames_chanel=0,
    n_frames=3,
    stride=-1,
)

In [ ]:
img, y, _ = dataset[2]

In [ ]:
img.size()

In [ ]:
idx = np.random.choice(len(dataset))
img, y, _ = dataset[idx]

if len(img.size()) == 4:
    plt.figure(figsize=(15, 5))
    for i, img_ in enumerate(img.numpy()):
        plt.subplot(1, img.size(0), i + 1)
        if img_.shape[0] == 3:
            plt.imshow(img_.transpose(1, 2, 0))
        else:
            plot_mask(img_[:3].transpose(1, 2, 0), img_[-1])
        plt.axis(False)
        
        plt.title(f"Target: {int(y.item())}")
    plt.show()
else:
    plt.figure(figsize=(5, 5))
    plt.imshow(img.numpy().transpose(1, 2, 0))
    plt.axis(False)
    plt.show()

## Model

In [ ]:
from util.torch import count_parameters

In [ ]:
model = define_model(
    'coatnet_rmlp_1_rw2_224',
    num_classes=2,
    num_classes_aux=11,
    n_channels=4,
    reduce_stride=False,
    drop_path_rate=0.,
    n_frames=3,
    use_gem=True,
    increase_stride=False,
    head_3d="lstm",
    verbose=1,
)

In [ ]:
x = torch.cat([img.unsqueeze(0)] * 1)
model = model.cuda()
x = x.cuda()
# pred, pred_aux = model(x)

In [ ]:
%%time
pred, pred_aux = model(x)

In [ ]:
pred.size(), pred_aux.size()
# y_patient.size(), y_img.size()

In [ ]:
optim = torch.optim.Adam(model.parameters())

loss = pred.mean() + pred_aux.mean()
loss.backward()

for n, p in model.named_parameters():
    if p.grad is None:
        print(n)

In [ ]:
y_p = torch.cat([y_patient.unsqueeze(0)] * 2)
y_i = torch.cat([y_img.unsqueeze(0)] * 2)

In [ ]:
loss = AbdomenLoss({
    "name": "image",
    "smoothing": 0,
    "aux_loss_weight": 0.5,
    "name_aux": "patient",
    "smoothing_aux": 0,
})

In [ ]:
pred.size(), y_i.size()

In [ ]:
loss(pred, pred_aux, y_i, y_p)

In [ ]:
# loss = PatientLoss()
# loss(pred_aux, y_p)

## Training

In [ ]:
class Config:
    """
    Parameters used for training
    """
    # General
    seed = 42
    verbose = 1
    device = "cuda"
    save_weights = True

    # Data
    resize = (512, 512)
    aug_strength = 1
    pos_prop = 0.1

    # k-fold
    k = 4
    folds_file = f"../input/folds_{k}.csv"
    selected_folds = [0]  # , 1, 2, 3]

    # Model
    name = "tf_efficientnet_b0"
    pretrained_weights = None # PRETRAINED_WEIGHTS[name]  # None
    
    num_classes = 2
    num_classes_aux = 0
    drop_rate = 0
    drop_path_rate = 0
    n_channels = 3
    reduce_stride = False
    replace_pad_conv = False
    use_gem = True

    # Training    
    loss_config = {
        "name": "image",
        "smoothing": 0,
        "activation": "sigmoid",
        "aux_loss_weight": 0,
        "name_aux": "patient",
        "smoothing_aux": 0,
        "activation_aux": "",
        "ousm_k": 0,  # todo ?
    }

    data_config = {
        "batch_size": 16,
        "val_bs": 16,
        "mix": "mixup",
        "mix_proba": 0.,
        "mix_alpha": 4.,
        "additive_mix": False,
        "num_classes": num_classes,
        "num_workers": 8,
    }

    optimizer_config = {
        "name": "AdamW",
        "lr": 5e-4,
        "warmup_prop": 0.,
        "betas": (0.9, 0.999),
        "max_grad_norm": 10.,
        "weight_decay": 0.,
    }

    epochs = 1

    use_fp16 = True

    verbose = 1
    verbose_eval = 50
    
    fullfit = False
    n_fullfit = 1

In [ ]:
DEBUG = True
log_folder = None
run = None

In [ ]:
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f"Logging results to {log_folder}")
    config_df = save_config(Config, log_folder + "config.json")
    create_logger(directory=log_folder, name="logs.txt")
#     run = init_neptune(Config, log_folder)

df_patient, df_img = prepare_data(DATA_PATH)
init_distributed(Config)

preds = k_fold(Config, df_patient, df_img, log_folder=log_folder, run=run)

Done ! 